<h1 align=center> Segmentation and clustering Neighborhoods in Tornoto </h1>

## introduction
In this project we will explore neighbourhood in Tornoto using Forsqure API.
first we'll webscrap the data from wikipedia and then using ForSquare API and folium map extract imporatnt information from this data. We will use k-means clusterning algorithm to do this task

## Table of Contents
  1. <a href="#item1">Part-1 </a>
     - import required libraries
     - extract and Explore Dataset from Wikipedia using Webscrapping
  2. <a href="#item2">Part-2 </a>
     - Read coordinates in dataframe from csv files
     - create final data frame
  3. <a href="#item3">Part-3 </a>
     - Use Geopy library to get coordinates of Toronto
     - Create a map of Toronto and add all neighborhoods on the same map
     - Define Foursqure credential
     - Explore Neighborhoods
     - Analyze each Neighborhood
     - Cluster Neighborhood¶
     
     
     
     
     
      
  
  
  
 

<h1 align='center'> Part-1 </h1>

### 1.1 import required libraries

In [ ]:
#import libraries
!pip install html5lib
!pip install lxml
!pip install geocoder
!pip install bs4
!pip install folium 
!pip install geopy
!pip install matplotlib  # Matplotlib and associated plotting modules

# import k-means from clustering stage
from sklearn.cluster import KMeans

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests
import geocoder

print("all libraries installed and imported succefully")

### 1.2 extract and Explore Dataset from Wikipedia using Webscrapping

In [6]:
# using webscrapping to extract data
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data= requests.get(url).text
beautiful_soup=BeautifulSoup(html_data,"html.parser")

# crete new data frame
df=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
df


# append data in data frame by webscrpping
for row in beautiful_soup.find("tbody").find_all("tr"):
    for col in row.find_all("td"):
        if col.span.text=='Not assigned':
            pass
        else:
            pcode=col.p.text[:3]
            borough=(col.span.text).split('(')[0]
            nhood=(((((col.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            df=df.append({'PostalCode':pcode,'Borough':borough,'Neighborhood':nhood},ignore_index=True)
        
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [7]:
df.shape


(103, 3)

<h1 align='center'> Part-2 </h1>

### 2.1 Read coordinates in dataframe from csv files

In [8]:
df_coord=pd.read_csv("Geospatial_Coordinates.csv")
df_coord.rename(columns= {'Postal Code':'PostalCode'},inplace=True)


### 2.2 merge both data frame to make final data frame which contain coordinates also

In [32]:
result=pd.merge(df,df_coord,on=['PostalCode'])
result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h1 align='center'> Part-3 </h1>
<h2 align='center'> Clustering and Segmenting </h2>

### 3.1 Use Geopy library to get coordinates of Toronto

In [10]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address='Toronto'
geolocator=Nominatim(user_agent='Tr_explore')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print("The geographical coordinates of Tronoto are: {},{}".format(latitude,longitude))

The geographical coordinates of Tronoto are: 43.6534817,-79.3839347


### 3.2 Create a map of Toronto and add all neighborhoods on the same map

In [11]:
# create map of Tronto
map_toronto= folium.Map(location=[latitude,longitude],zoom_start=13)

#add neghborhood on toronto map

for lat,lng,borough,nhood in zip(result['Latitude'],result['Longitude'],result['Borough'],result['Neighborhood']):
        label="{},{}".format(nhood,borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto


above map is too congested, so for illustration purpose We'll use small set of Data
Let's simplify the above map and consider only those neighborhood whose Borough name conatains "Toronto"

In [33]:
result.head()
tr_data=result[ result['Borough'].str.contains('Toronto')].reset_index(drop=True)
tr_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [13]:
# create map of Tronto
map_toronto= folium.Map(location=[latitude,longitude],zoom_start=13)

#add neghborhood on toronto map

for lat,lng,borough,nhood in zip(tr_data['Latitude'],tr_data['Longitude'],tr_data['Borough'],tr_data['Neighborhood']):
        label="{},{}".format(nhood,borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

Now we are going to use Foursqure API to explore the neighborhood and explore them

###  3.3Define Foursqure credential

In [14]:
CLIENT_ID = '15VSS0FQMGGTRNLEKHI1SFVW1CCLK12MDZ3E4GZKWJHNQHO1' # your Foursquare ID
CLIENT_SECRET = 'FLYBN5UVHBPC1DZGXGJWCI1K53K3PJHVCYXGMQJVE4FJPQOC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 15VSS0FQMGGTRNLEKHI1SFVW1CCLK12MDZ3E4GZKWJHNQHO1
CLIENT_SECRET:FLYBN5UVHBPC1DZGXGJWCI1K53K3PJHVCYXGMQJVE4FJPQOC


Let's explore the first neighborhood from our data frame

In [15]:
nhood_name=tr_data.loc[0,'Neighborhood'].split(',')[0].strip()
nhood_name

'Regent Park'

Get the neighborhood latitude and longitude

In [16]:
nhood_lat= tr_data.loc[0,'Latitude']
nhood_lng= tr_data.loc[0,'Longitude']
print("selected neighborhood is {} and its Coordinate are: {},{}".format(nhood_name,nhood_lat,nhood_lng))

selected neighborhood is Regent Park and its Coordinate are: 43.6542599,-79.3606359


### Let's get top 100 venues in Regent Park within a radius of 300 metre

In [17]:
LIMIT=100
radius=300
url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    nhood_lat, 
    nhood_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=15VSS0FQMGGTRNLEKHI1SFVW1CCLK12MDZ3E4GZKWJHNQHO1&client_secret=FLYBN5UVHBPC1DZGXGJWCI1K53K3PJHVCYXGMQJVE4FJPQOC&v=20180605&ll=43.6542599,-79.3606359&radius=300&limit=100'

In [34]:
results=requests.get(url).json()
#results


In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venue=results['response']['groups'][0]['items']
nearby_venues=json_normalize(venue)
nearby_venues

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.shape)
nearby_venues.head()

(13, 4)


<ipython-input-20-6f98163cd504>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues=json_normalize(venue)


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


### 3.4 Explore Neighborhoods 

define a function getNearbyVenue which takes location of Neighborhood and create data frame of max 100 venues which lies within 500 meter radius of that neighborhood

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

toronto_venue is data frame of all the venues details for all neighborhood which contains word 'Toronto'

In [22]:
toronto_venue=getNearbyVenues(tr_data['Neighborhood'],tr_data['Latitude'],tr_data['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [23]:
print(toronto_venue.shape)
toronto_venue.head()


(1572, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


#### Count No. of values returned for each neighborhood

In [ ]:
toronto_venue.groupby("Neighborhood").count()

Let's try to find out no. of venues for each category in neighborhood of Toronto <br>
We can sort them is descending order and we can find that coffe shop is maximum followed by cafe an then Restaurant

In [ ]:
toronto_venue.groupby("Venue Category").count().sort_values(by='Venue',ascending=False)

### 3.5 Analyze each Neighborhood

In [26]:

# one hot encoding
toronto1 = pd.get_dummies(toronto_venue[['Venue Category']], prefix="", prefix_sep="")
toronto1.drop("Neighborhood",inplace=True,axis=1)
# add neighborhood column back to dataframe
toronto1['Neighborhood'] = toronto_venue['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto1.columns[-1]] + list(toronto1.columns[:-1])
toronto1 = toronto1[fixed_columns]
toronto1.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_grouped=toronto1.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0,0.071429,0.071429,0.142857,0.142857,0.142857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,Central Bay Street,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.016129,0.016129
4,Christie,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


Let's find out the top 10 venues for each Neigborhood

In [28]:
# crete a function which return top n common venue
def top_n_venue(row,n):
    venue=row[1:]
    venue_sorted=venue.sort_values(ascending=False)
    return venue_sorted.index.values[0:n]

In [29]:
n=10
columns=['Neighborhood',1,2,3,4,5,6,7,8,9,10]

toronto_venue_sorted=pd.DataFrame(columns=columns)
toronto_venue_sorted['Neighborhood']=toronto_grouped['Neighborhood']
for i in range(toronto_grouped.shape[0]):
    toronto_venue_sorted.iloc[i,1:]=top_n_venue(toronto_grouped.iloc[i,:],n)
toronto_venue_sorted.head()


,Neighborhood,1,2,3,4,5,6,7,8,9,10
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Pharmacy,Beer Bar,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Italian Restaurant,Burrito Place,Stadium,Furniture / Home Store,Bar,Bakery
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Airport Food Court,Harbor / Marina,Boutique,Sculpture Garden,Boat or Ferry
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Salad Place,Bank,Bubble Tea Shop,Burger Joint,Thai Restaurant
4,Christie,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Nightclub,Athletics & Sports,Restaurant,Coffee Shop,Candy Store


### 3.6 Cluster Neighborhood

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 1,
       0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:

toronto_venue_sorted.insert(0,'Cluster Labels',kmeans.labels_)
toronto_venue_sorted
final=tr_data
final=final.join(toronto_venue_sorted.set_index('Neighborhood'),on='Neighborhood')
final

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1,2,3,4,5,6,7,8,9,10
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,Café,Restaurant,Pub,Event Space,Chocolate Shop
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Hotel,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Bubble Tea Shop,Lingerie Store
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Gastropub,Italian Restaurant,Lingerie Store,Seafood Restaurant,Moroccan Restaurant,Cosmetics Shop,Cheese Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Adult Boutique,Movie Theater,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Pharmacy,Beer Bar,Hotel
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Salad Place,Bank,Bubble Tea Shop,Burger Joint,Thai Restaurant
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Nightclub,Athletics & Sports,Restaurant,Coffee Shop,Candy Store
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Gym,Hotel,Clothing Store,Thai Restaurant,Deli / Bodega,Cosmetics Shop,Sushi Restaurant
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Pharmacy,Bakery,Post Office,Bar,Music Venue,Supermarket,Café,Middle Eastern Restaurant,Liquor Store,Grocery Store
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,0,Convenience Store,Coffee Shop,Park,Metro Station,Adult Boutique,Museum,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant


In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final['Latitude'], final['Longitude'], final['Neighborhood'], final['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters